In [4]:
# Libraries
import time
import pandas as pd    
# ------------- # 
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
# ------------- # 
from webdriver_manager.chrome import ChromeDriverManager

# library for interacting with the operating system
import os
# ------------- # 
from bs4 import BeautifulSoup
import requests
# ------------- # 
from langdetect import detect
# ------------- # 
import random

 # ------------- # 
from datetime import date

In [5]:
# installing driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [6]:
# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)

In [7]:
# Accept cookies
driver.find_element(By.XPATH,'''//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]''').click()


In [8]:
# User Credentials
# Reading txt file where we have our user credentials
with open('user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]

user_name = user_credentials[0] # First line
password = user_credentials[1] # Second line
# email=driver.find_element(By.XPATH,('//*[@id="username"]').send_keys(user_name))
# passw= driver.find_element(By.XPATH,('//*[@id="password"]').send_keys(password))
time.sleep(1)

email=driver.find_element(By.XPATH,('//*[@id="username"]'))
passw= driver.find_element(By.XPATH,('//*[@id="password"]'))
email.send_keys(user_name)
passw.send_keys(password)
#end_keys(Keys.ENTER)


In [9]:
# Login button
driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(5)

In [10]:
# Jobs page
driver.find_element(By.XPATH,'//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(3)
# Go to search results directly

## JUNIOR DATA -> NETHERLANDS
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3365495151&f_E=2&geoId=102890719&keywords=%22junior%20data%22&location=Netherlands&refresh=true")
time.sleep(1)



In [11]:
# go to the end of the page in order for all the elements to be loaded

job_viewer = driver.find_element(By.CSS_SELECTOR, "a[class^='disabled ']")
job_viewer.send_keys(Keys.END)

In [12]:
## CHECK HOW MANY PAGES
num_pages = driver.find_elements(By.CSS_SELECTOR, "li[data-test-pagination-page-btn]")

In [13]:
pages=[page.text for page in num_pages]
pages

['1', '2', '3']

In [14]:
# Get all links for these offers
links = []
# Navigate 13 pages
num_pages=len(pages)
#print(len)
print('Links are being collected now.')
try: 
    print('here1')
    for page in range(2,num_pages+1):
        print(num_pages)
        
        time.sleep(random.randint(1,4))
        jobs_block = driver.find_element(By.CLASS_NAME,'scaffold-layout__list-container')
        jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
        #print("jobs_block",jobs_block)
        #print(jobs_list)
    
        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME,'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)
        
        print(f'Collecting the links in the page: {page-1}')
        # go to next page:
        print('going to next page')
        driver.find_element(By.XPATH,f"//button[@aria-label='Page {page}']").click()
        time.sleep(random.randint(1,4))
except:
    pass
print('Found ' + str(len(links)) + ' links for job offers')

Links are being collected now.
here1
3
going to next page
3
going to next page
Found 50 links for job offers


In [15]:
num_pages

3

In [16]:
# # Get all links for these offers
# links = []
# # Navigate 13 pages
# num_pages=len(pages)
# #print(len)
# print('Links are being collected now.')

# #print('here1')
# try: 
#     for page in range(2,num_pages+2):
#         print(page)

#         time.sleep(random.randint(1,4))
#         jobs_block = driver.find_element(By.CLASS_NAME,'scaffold-layout__list-container')
#         jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
#         #print("jobs_block",jobs_block)
#         #print(jobs_list)

#         for job in jobs_list:
#             all_links = job.find_elements(By.TAG_NAME,'a')
#             for a in all_links:
#                 if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
#                     links.append(a.get_attribute('href'))
#                 else:
#                     pass
#             # scroll down for each job element
#             driver.execute_script("arguments[0].scrollIntoView();", job)

#         print(f'Collecting the links in the page: {page-1}')
#         # go to next page:
#         print('going to next page')
#         driver.find_element(By.XPATH,f"//button[@aria-label='Page {page}']").click()
#         time.sleep(random.randint(1,4))

# except:
#     pass
# print('Found ' + str(len(links)) + ' links for job offers')

In [17]:
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = [] 
job_desc = []
lan_list=[]

i = 0
j = 1

In [18]:
# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i in range(len(links)):
#for i in range(2):
    try:   
        driver.get(links[i])
        i=i+1
        time.sleep(random.randint(1,3))
        # Click See more.
        driver.find_element(By.CLASS_NAME,"artdeco-card__actions").click()
        time.sleep(2)
    except:
        pass


    try:

        job_titles.append(driver.find_element(By.TAG_NAME,"h1").text)
        print('here3')
        company_names.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__company-name").text)
        company_locations.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__bullet").text)
        work_methods.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__workplace-type").text)
        post_dates.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__posted-date").text)
        work_times.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-insight").text)
        print(f'Scraping the Job Offer {j} DONE.')
        j+= 1
    except:
        pass

    job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')

    try:
        for description in job_description:
            job_text = description.find_element(By.CLASS_NAME,"jobs-box__html-content").text
            lan=detect(job_text)
            job_desc.append(job_text)
            lan_list.append(lan)
            print(f'Scraping the Job Offer {j}')
            time.sleep(random.randint(1,3))

    except:
        pass

        

      

Visiting the links and collecting information just started.
here3
Scraping the Job Offer 1 DONE.
Scraping the Job Offer 2
here3
Scraping the Job Offer 2 DONE.
Scraping the Job Offer 3
here3
Scraping the Job Offer 3 DONE.
Scraping the Job Offer 4
here3
Scraping the Job Offer 4 DONE.
Scraping the Job Offer 5
here3
Scraping the Job Offer 5 DONE.
Scraping the Job Offer 6
here3
Scraping the Job Offer 6 DONE.
Scraping the Job Offer 7
here3
Scraping the Job Offer 7 DONE.
Scraping the Job Offer 8
here3
Scraping the Job Offer 8 DONE.
Scraping the Job Offer 9
here3
Scraping the Job Offer 9 DONE.
Scraping the Job Offer 10
here3
Scraping the Job Offer 10 DONE.
Scraping the Job Offer 11
here3
Scraping the Job Offer 11 DONE.
Scraping the Job Offer 12
here3
Scraping the Job Offer 12 DONE.
Scraping the Job Offer 13
here3
Scraping the Job Offer 13 DONE.
Scraping the Job Offer 14
here3
Scraping the Job Offer 14 DONE.
Scraping the Job Offer 15
here3
Scraping the Job Offer 15 DONE.
Scraping the Job Offer 

In [19]:
# Creating the dataframe 
df = pd.DataFrame(list(zip(job_titles,company_names,
                    company_locations,job_desc,lan_list,links)),
                    columns =['job_title', 'company_name',
                           'company_location','job_desc','lan','link'])

In [20]:
df

,job_title,company_name,company_location,job_desc,lan,link
0,Education SaaS – Junior Quality Engineer / Tes...,Vivid Resourcing Ltd,Utrecht Area,About the job\nEducation SaaS – Junior Quality...,en,https://www.linkedin.com/jobs/view/3434484880/...
1,Junior Business Consultant,Itility,"Eindhoven, North Brabant, Netherlands",About the job\nAre you at the start of your ca...,en,https://www.linkedin.com/jobs/view/3365495151/...
2,Software Engineer (Junior or Senior),Samotics,"Leiden, South Holland, Netherlands",About the job\nJob Description\n\nAbout Samoti...,en,https://www.linkedin.com/jobs/view/3331733065/...
3,"Junior Project Manager (Enablon, The Hague)",Wolters Kluwer,"The Hague, South Holland, Netherlands",About the job\nDo you want to implement softwa...,en,https://www.linkedin.com/jobs/view/3339932483/...
4,IT & Strategy Talent Programme - Junior Data E...,Vattenfall,"Amsterdam, North Holland, Netherlands",About the job\nNow you can shape the path towa...,en,https://www.linkedin.com/jobs/view/3432947895/...
5,Junior Finance Controller,Bright River,"Haarlem, North Holland, Netherlands",About the job\nJunior Finance Controller\n\nWh...,en,https://www.linkedin.com/jobs/view/3434476710/...
6,Junior Financial Accountant,Vesper,"Amsterdam, North Holland, Netherlands",About the job\nWho we are\n\nVesper is the fas...,en,https://www.linkedin.com/jobs/view/3386020984/...
7,Junior Account Manager,Enreach,"Almere, Flevoland, Netherlands",About the job\nJUNIOR ACCOUNT MANAGER/INSIDE S...,nl,https://www.linkedin.com/jobs/view/3434992010/...
8,Jr. Email Specialist,Orderchamp,"Amsterdam, North Holland, Netherlands",About the job\nHigh-growth start-up looking fo...,en,https://www.linkedin.com/jobs/view/3340595679/...
9,Junior data engineer (traineeship),Berenschot,"Utrecht, Netherlands",About the job\nBerenschot zoekt een junior dat...,nl,https://www.linkedin.com/jobs/view/3422415797/...


In [21]:
# EXPORT DATAFRAME TO EXCEL
date=date.today()
filename=f"Excel_files/linkedin_juniordata_{date}.xlsx"
df.to_excel(filename,index=False)

In [22]:
df=pd.read_excel("Excel_files\linkedin_juniordata_2023-01-11.xlsx")

In [23]:
df

,job_title,company_name,company_location,work_method,post_date,work_time,job_desc,lan
0,Junior Data & Informatie Consultant (Asset Man...,Royal HaskoningDHV,"Nijmegen, Gelderland, Netherlands",Hybrid,5 hours ago,Full-time · Entry level,About the job\nAls Junior Data & Informatie Co...,nl
1,"JUNIOR DATA SCIENTIST - Dubai, UAE",Cobblestone Energy,"Eindhoven, North Brabant, Netherlands",On-site,3 weeks ago,Full-time · Entry level,"About the job\nLocation: Dubai, UAE (We provid...",en
2,"JUNIOR DATA SCIENTIST - Dubai, UAE",Cobblestone Energy,"Eindhoven, North Brabant, Netherlands",On-site,3 weeks ago,Full-time · Entry level,"About the job\nLocation: Dubai, UAE (We provid...",en
3,Junior Data & Informatie Consultant (Asset Man...,Royal HaskoningDHV,"Rotterdam, South Holland, Netherlands",Hybrid,3 weeks ago,Full-time · Entry level,About the job\nAls Junior Data & Informatie Co...,nl
4,IT Traineeship - Data,Devoteam,"Amsterdam, North Holland, Netherlands",On-site,1 week ago,Full-time · Entry level,About the job\n\nVacatureomschrijving\n\nSta j...,nl
5,"Junior Data Engineer, Limburg, Maastricht",Precedence,"Maastricht, Limburg, Netherlands",On-site,1 week ago,Full-time · Entry level,About the job\nThis job is sourced from a job ...,nl
6,Cognitive & Data Analist.,True Legends IT Professionals,"Utrecht, Utrecht, Netherlands",On-site,1 month ago,Full-time · Entry level,About the job\nSamenvatting\n\nBen jij een jun...,nl
7,Junior Data Specialist,Favorite Gifts,"Tiel, Gelderland, Netherlands",On-site,1 week ago,Full-time · Entry level,About the job\nThis job is sourced from a job ...,nl
8,Data Scientist | Amsterdam,CareerValue,"Amsterdam, North Holland, Netherlands",On-site,2 weeks ago,Full-time · Entry level,About the job\nThis job is sourced from a job ...,nl
9,jr. Data & Reporting Analyst,Dechra Pharmaceuticals Manufacturing,"’s-Hertogenbosch, North Brabant, Netherlands",On-site,3 days ago,Full-time · Entry level,About the job\nThis job is sourced from a job ...,nl
